In [ ]:
import requests
import numpy as np
!pip install "tensorflow>=1.14.0,<2.0"
!pip install "tensorflow-gpu>=1.14.0,<2.0"
!pip install  ampligraph

In [3]:
import ampligraph
import numpy as np
ampligraph.__version__
import pandas as pd
from ampligraph.datasets import load_from_ntriples
from ampligraph.datasets import load_from_csv

In [4]:
Y=load_from_csv('/content','1100data.csv',sep=',',add_reciprocal_rels=False)

In [5]:
print(Y)

[['Common symptoms' 'include' 'fever']
 ['Management' 'involves' 'treatment']
 ['The World Health Organization' 'declared' 'outbreak']
 ...
 ['Recent studies' 'shown' 'Nocardia infections']
 ['individuals' 'increased' 'rates']
 ['autoimmune diseases' 'have' 'association']]


In [6]:
Y.shape

(1078, 3)

In [7]:
entities = np.unique(np.concatenate([Y[:, 0], Y[:, 2]]))
entities.size

1792

In [8]:
relations = np.unique(Y[:, 1])
relations.size

489

In [9]:
from ampligraph.evaluation import train_test_split_no_unseen 

Y_train, Y_test = train_test_split_no_unseen(Y, test_size=25) 

In [10]:
print('Train set size: ', Y_train.shape)
print('Test set size: ', Y_test.shape)

Train set size:  (1053, 3)
Test set size:  (25, 3)


In [11]:
print(Y_test[:25])

[['common symptoms' 'include' 'cough']
 ['France' 'banned' 'use']
 ['Risk factors' 'include' 'exposure']
 ['Management' 'includes' 'treatment']
 ['Symptoms' 'include' 'shortness']
 ['Causes' 'include' 'sepsis']
 ['Signs' 'include' 'shortness']
 ['Adenoviruses' 'cause' 'gastroenteritis']
 ['air pollution' 'causes' 'deaths']
 ['Air pollution' 'increases' 'risk']
 ['people' 'develop' 'symptoms']
 ['Complications' 'include' 'lung cancer']
 ['Complications' 'include' 'lung abscess']
 ['Astroviruses' 'cause' 'infection']
 ['symptoms' 'include' 'diarrhea']
 ['Symptoms' 'include' 'diarrhea']
 ['Mamastroviruses' 'cause' 'diseases']
 ['Symptoms' 'include' 'episodes']
 ['Individuals' 'experience' 'symptoms']
 ['People' 'have' 'higher rates']
 ['BBC News Online' 'had' 'number']
 ['Studies' 'shown' 'correlation']
 ['bats' 'consume' 'large amounts']
 ['Bats' 'use' 'variety']
 ['individuals' 'increased' 'rates']]


In [ ]:
print(Y_train[:200])

[['Common symptoms' 'include' 'fever']
 ['Management' 'involves' 'treatment']
 ['The World Health Organization' 'declared' 'outbreak']
 ['person' 'is' 'symptoms']
 ['minority' 'develop' 'noticeable symptoms']
 ['Cardiovascular complications' 'include' 'heart failure']
 ['Sputum' 'carry' 'large amounts']
 ['indicative' 'suggest' 'underlying immunopathology']
 ['people' 'have' 'classical serum biomarkers']
 ['The US Food and Drug Administration' 'approved' 'test']
 ['guidelines' 'recommend' 'medication']
 ['Face coverings' 'limit' 'volume']
 ['the United States Environmental Protection Agency' 'maintains' 'list']
 ['Intensivists' 'compiled' 'treatment recommendations']
 ['availability' 'affect' 'mortality']
 ['study' 'reported' 'earliest date']
 ['Official publications' 'reported' 'earliest onset']
 ['Wuhan Central Hospital' 'sent'
  'bronchoalveolar lavage fluid BAL sample']
 ['the National Health Commission of China' 'issued' 'notice']
 ['China' 'reported' 'nearly 140 new cases']
 ['It

In [ ]:
from ampligraph.latent_features import DistMult

In [ ]:
model = DistMult(batches_count=10, 
                seed=0, 
                epochs=20, 
                k=10, 
                eta=10,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(Y_train, early_stopping = False)

Average Loss:   2.368235: 100%|██████████| 20/20 [00:00<00:00, 38.10epoch/s]


In [ ]:
model.get_embeddings(['The World Health Organization','outbreak'], embedding_type='entity')

array([[-0.13353874, -0.05063289, -0.12340237, -0.08606508,  0.09882136,
         0.1034347 , -0.14308397,  0.0130221 , -0.08697779, -0.1022373 ],
       [-0.10325506, -0.03760489, -0.10346974,  0.08304343, -0.10251775,
        -0.0799965 , -0.13056728,  0.01013351, -0.08741832, -0.10676534]],
      dtype=float32)

In [ ]:
model.get_embeddings(['common symptoms','cough'], embedding_type='entity')

array([[ 0.12775518,  0.0465753 ,  0.11594734, -0.11839459,  0.08088872,
         0.07267105,  0.17203316, -0.08184921, -0.09523217,  0.03592303],
       [-0.13099752, -0.13748986,  0.09453399,  0.0492803 , -0.08164638,
        -0.11327812, -0.01196939,  0.13249981,  0.16907424,  0.11841266]],
      dtype=float32)

In [ ]:
positives_filter = Y

In [ ]:
from ampligraph.latent_features import save_model, restore_model

In [ ]:
save_model(model, './covid_model.pkl')

In [ ]:
from ampligraph.evaluation import evaluate_performance

In [ ]:
from ampligraph.discovery import find_duplicates

In [ ]:
entities = np.unique(Y_train[:, 2])
dups, _ = find_duplicates(entities, model, mode='entity', tolerance=0.4)
print(list(dups)[:3])

[frozenset({'advanced tertiary economy', 'conspicuous collar', 'drop', 'neuronal necrosis', 'cardiac pacing', 'trials', 'disparities', 'airway inflammation', '% difference', 'at least 50% European ancestry', 'bronchiectasis', '293 London residents', 'tighter regulations', 'tax relief', 'greater contact', 'three of the original nine patents on the computer on which all PCs', 'about 45%', 'lung cancer', 'dust mites', 'similar blood', 'red blood cells', 'seizures', 'reactions', 'captives', 'bronchoconstriction', 'two defendants', 'oil war', 'severe exacerbation', 'multi - user editing', 'literature', 'early late phases', 'app', 'mainstream Republicans', 'James Armistead', 'seat', 'low', '742 million fiscal stimulus program', 'driving protest', 'research based senior Medical Doctorate', 'permanent colostomies', 'new onset Crohn disease', 'Langston Hughes', 'positive stranded RNA virus replication model', 'bug bounty program', 'atypical symptoms:.No response', 'egg', 'waste products', 'foll

In [ ]:
from ampligraph.discovery import query_topn

In [ ]:
query_topn(model, top_n=5,
            head=None, relation='include', tail='cough',
           ents_to_consider=None, rels_to_consider=None)

(array([['Symptom', 'include', 'cough'],
        ['Biomarkers', 'include', 'cough'],
        ['American Society of Microbiology).Endospores', 'include',
         'cough'],
        ['plasma life', 'include', 'cough'],
        ['Human activities', 'include', 'cough']], dtype='<U70'),
 array([0.03469571, 0.02868232, 0.02791877, 0.0276491 , 0.02664187],
       dtype=float32))

In [ ]:
ranks = evaluate_performance(Y_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 25/25 [00:00<00:00, 310.00it/s]


In [ ]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
mr = mr_score(ranks)
mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))
hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.01
MR: 1075.90
Hits@10: 0.02
Hits@3: 0.00
Hits@1: 0.00


In [ ]:
from ampligraph.utils import create_tensorboard_visualizations

In [ ]:
create_tensorboard_visualizations(model,r"./")

In [ ]:
X_unseen = np.array([
    ['guidelines', 'recommend', 'medication'],
  ['black musician', 'wears', 'royal livery'],
   ['Common effects', 'include', 'chest infections'],
 #   ['pandemic', 'include', 'global social economic disruption']

])

In [ ]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))

In [ ]:
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)

100%|██████████| 3/3 [00:00<00:00, 62.12it/s]


In [ ]:
scores = model.predict(X_unseen)

In [ ]:
from scipy.special import expit
probs = expit(scores)

In [ ]:
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")

,statement,rank,score,prob
2,Common effects include chest infections,1700,0.000601,0.500150
1,black musician wears royal livery,9,0.009461,0.502365
0,guidelines recommend medication,2,0.015268,0.503817
